In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('demandd.csv',sep=';')

/tmp/ipykernel_14102/800761036.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('demandd.csv',sep=';')


In [3]:
defecated=df[df['Dobowa prognoza zapotrzebowania KSE']=='-']
df=df[df['Rzeczywiste zapotrzebowanie KSE']!='-']
df=df[df['Dobowa prognoza zapotrzebowania KSE']!='-']

df=df.rename(columns={'Dobowa prognoza zapotrzebowania KSE':'PSEforecast','Rzeczywiste zapotrzebowanie KSE':'Load'})
df['Load']=df.Load.transform(lambda x: x.replace(',','.')).astype(float)
df['PSEforecast']=df.PSEforecast.astype(float)
df['Data']=df.Data.astype(str)
df=df.drop_duplicates()
df.reset_index(inplace=True)

In [4]:
df['Godz.'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '2A'], dtype=object)

In [5]:
df.loc[df['Godz.']=='2A','Godz.']=2
df['Godz.']=(df['Godz.'].astype(int)-1).astype(str)

In [6]:
df['Time']=pd.to_datetime(df['Data']+df['Godz.'],format='%Y%m%d%H',errors='coerce')

In [7]:
indexes_to_change=[7079, 15931, 24667, 33403, 42139, 50875, 59777, 68513, 77249]
indexes_to_change2=[x-1 for x in indexes_to_change]





In [8]:
df['error']=0
for i in indexes_to_change:
    df.loc[i,'error']=1

In [9]:
df['Time']=df.Time.dt.tz_localize('Europe/Warsaw',ambiguous=df.error)


In [10]:
df['Timex']=0
hour=pd.Timedelta(1,unit='h')
df.loc[indexes_to_change,'Timex']=df.loc[indexes_to_change,'Time'].dt.tz_convert('+01:00')+hour
df.loc[indexes_to_change2,'Timex']=df.loc[indexes_to_change2,'Time'].dt.tz_convert('+01:00')+hour


/tmp/ipykernel_14102/4265865675.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2015-10-25 02:00:00+01:00', '2016-10-30 02:00:00+01:00',
 '2017-10-29 02:00:00+01:00', '2018-10-28 02:00:00+01:00',
 '2019-10-27 02:00:00+01:00', '2020-10-25 02:00:00+01:00',
 '2021-10-31 02:00:00+01:00', '2022-10-30 02:00:00+01:00',
 '2023-10-29 02:00:00+01:00']
Length: 9, dtype: datetime64[ns, UTC+01:00]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[indexes_to_change,'Timex']=df.loc[indexes_to_change,'Time'].dt.tz_convert('+01:00')+hour


In [11]:
df.loc[df.Timex==0,'Timex']=df.Time
df.drop(['index','Time','error','Data'],axis=1,inplace=True)
df.rename(columns={'Godz.':'Hour','Timex':'Time'},inplace=True)


In [12]:
df['Time']=pd.to_datetime(df.Time,utc=True)

In [13]:
df.set_index('Time',drop=True,inplace=True)


In [14]:
dff=df.reindex(pd.date_range(df.index.min(),df.index.max(),freq=hour))
dff.drop(dff.columns,axis=1,inplace=True)
df=df.join(dff, how='right', rsuffix='_left')

In [17]:


df.Load.interpolate(method='linear',axis=0,inplace=True)
df['PSEforecast'].interpolate(method='linear',axis=0,inplace=True)
df[df.Hour.isna()==True]


,Hour,PSEforecast,Load
2015-03-10 20:00:00+00:00,NaN,20634.500000,20641.681250
2015-03-10 21:00:00+00:00,NaN,19317.000000,19237.837500
2015-03-10 22:00:00+00:00,NaN,17999.500000,17833.993750
2015-04-13 22:00:00+00:00,NaN,17458.000000,17158.354000
2015-04-13 23:00:00+00:00,NaN,17416.000000,17116.508000
...,...,...,...
2016-10-04 19:00:00+00:00,NaN,17673.076923,17346.683077
2016-10-04 20:00:00+00:00,NaN,17265.384615,16956.751385
2016-10-04 21:00:00+00:00,NaN,16857.692308,16566.819692
2021-02-10 21:00:00+00:00,NaN,23366.666667,22628.029333


In [30]:
df.drop('Hour',axis=1,inplace=True)

In [32]:
df.to_csv('Load.csv')